In [3]:
import pandas as pd
import numpy as np
from tqdm import tqdm
pd.set_option("display.max_columns", None)

# Offerta Luiss
- Contiene la lista di *professioni per corso di studi*.

- Il codice della professione fa riferimento alla **classificazione CP2021** dell'ISTAT.

- Abbiamo quindi bisogno di transcodificarlo nel **codice ISCO-08** presente nel database OJA

In [4]:
offerta = pd.read_excel("./data/offerta_luiss/offerta_formativa_luiss.xlsx")
offerta.columns = ['percorso', 'facolta', 'corso_studi_id', 'corso_studi_desc', 'cp2021', 'cp2021_desc']
print("Numero di righe del file:", offerta.shape[0])
print("cp2021 univoci:", offerta.cp2021.nunique())
offerta.head(2)

Numero di righe del file: 118
cp2021 univoci: 54


,percorso,facolta,corso_studi_id,corso_studi_desc,cp2021,cp2021_desc
0,LAUREE TRIENNALI,Global Law,L-14,Scienze dei servizi giuridici,2.5.2.2.2,Esperti legali in enti pubblici
1,LAUREE TRIENNALI,Global Law,L-14,Scienze dei servizi giuridici,3.4.5.4.0,Tecnici dei servizi di sicurezza privati e pro...


# Raccordo professioni ISTAT CP2021 vs ISCO-08
Il raccordo è disponibile solo per il livello 4 dei codici ISCO-08

In [28]:
raccordo_istat_isco4 = pd.read_parquet("./data/classificazioni/esco_raccordo_istat_isco4.parquet")

# recuperati manualmente
# Non sono sicuro solo del primo, gli altri sono stati invece recuperati tramite uguale descrizione.
raccordo_manuale = pd.DataFrame([{'cp2021': '3.3.2.6.1', 'occupation4d_id': 'OC3312'} 
                                 , {'cp2021': '2.7.1.1.2', 'occupation4d_id': 'OC2511'}
                                 , {'cp2021': '2.7.1.1.2', 'occupation4d_id': 'OC2519'}
                                 , {'cp2021': '2.7.1.1.1', 'occupation4d_id': 'OC2511'}
                                 , {'cp2021': '2.7.1.1.1', 'occupation4d_id': 'OC2120'}
                                 , {'cp2021': '2.7.1.1.1', 'occupation4d_id': 'OC2131'}
                                 , {'cp2021': '2.7.1.1.1', 'occupation4d_id': 'OC2166'}
                                 , {'cp2021': '2.7.1.1.1', 'occupation4d_id': 'OC2512'}
                                 , {'cp2021': '2.7.1.1.1', 'occupation4d_id': 'OC2514'}
                                 , {'cp2021': '2.7.1.1.1', 'occupation4d_id': 'OC2519'}
                                 , {'cp2021': '2.7.1.1.3', 'occupation4d_id': 'OC2513'}
                                 , {'cp2021': '2.7.1.1.3', 'occupation4d_id': 'OC2514'}
                                 , {'cp2021': '2.7.2.1.3', 'occupation4d_id': 'OC2511'}
                                 , {'cp2021': '2.7.2.1.3', 'occupation4d_id': 'OC2522'}
                                 , {'cp2021': '2.7.2.1.2', 'occupation4d_id': 'OC2521'}
                                ])

raccordo_istat_isco4 = pd.concat([raccordo_istat_isco4, raccordo_manuale])[['cp2021', 'occupation4d_id']].drop_duplicates()
raccordo_istat_isco4.head(3)

,cp2021,occupation4d_id
0,1.1.1.1.0,OC1111
5,1.1.1.2.0,OC1111
6,1.1.1.3.0,OC1111


In [32]:
offerta_isco08_liv4 = offerta.merge(raccordo_istat_isco4, how='left', on='cp2021')
print("Numero di righe del file:", offerta_isco08_liv4.shape[0])
print("cp2021 univoci:", offerta_isco08_liv4.cp2021.nunique())
offerta_isco08_liv4.to_parquet("./data/offerta_luiss/offerta_formativa_isco4.parquet")

Numero di righe del file: 294
cp2021 univoci: 54
